In [1]:
import mwu_measures
import pandas as pd
import mwu_measures.preprocessing_corpus
import mwu_measures.processing_corpus
from mwu_measures.corpus_helper import Fetcher
from mwu_measures.corpus import Corpus
import duckdb
from rich.progress import Progress

In [2]:

mwu_examples = pd.read_csv('MultiwordExpression_Concreteness_Ratings.csv')
mwu_examples['length'] = mwu_examples['Expression'].apply(lambda x: len(x.split()))
mwu_examples = mwu_examples.loc[(mwu_examples['length'] == 2) | (mwu_examples['length'] == 3)]
mwu_examples['Expression'] = mwu_examples['Expression'].apply(lambda x: x.lower())


In [3]:
# mwu_measures.processing_corpus.make_processed_corpus('coca', 'mwu_measures/corpora/coca_texts/', chunk_size=10, verbose=True)
# wrapper = lp(mwu_measures.processing_corpus.get_processed_corpus)
# this_corpus = mwu_measures.processing_corpus.get_processed_corpus('bnc', 'small_corpus.txt', chunk_size=10000000, verbose=False)
# this_corpus = mwu_measures.processing_corpus.make_processed_corpus(test_corpus=True, threshold=0)
# lp.print_stats()
# ngram_selection = [ngram.split() for ngram in ngram_selection]
# ngram_chunks = np.array_split(ngram_selection, 100)
bigrams = [['b', 'd'], ['c', 'b'], ['a', 'c']]
# helper = Fetcher(this_corpus)
# 340 seconds for all _acad_. Not bad, not the best.

In [ ]:
test_corpus = Corpus("test")
helper = Fetcher(test_corpus)

In [5]:
test_corpus.create_query(bigrams, 'ug_1', 'ug_2')

In [ ]:
test_corpus.get_ngram_scores('ug_1', 'ug_2', 2)['raw']

In [ ]:
this_corpus = Corpus("coca")
helper = Fetcher(this_corpus)


In [8]:
ngrams = mwu_examples['Expression'].tolist()
# ngrams = [[ngram[0] + ' ' + ngram[1], ngram[2]] if len(ngram) == 3 else ngram for ngram in ngrams]

In [ ]:
x = helper.get_score_batch(ngrams)


In [ ]:
x[0]

In [ ]:

x[0][~x[0]['ngram_length'].isna()].drop_duplicates().sort_values(by=['mwu_score'], ascending=False).iloc[0:100]

In [2]:
human_data_1 = pd.read_csv('test.csv', usecols=['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'])
human_data_2 = pd.read_csv('train.csv', usecols=['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'])
human_data_3 = pd.read_csv('valid.csv', usecols=['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'])
human_data = pd.concat([human_data_1, human_data_2, human_data_3]).reset_index(drop=True)

In [159]:
human_data['utterance'] = human_data['utterance'].str.replace('_comma_', ' , ')
human_data['prompt'] = human_data['prompt'].str.replace('_comma_', ' , ')
human_data['utterance'] = human_data['utterance'].str.replace('n t ', "n't ", regex=False)
human_data['prompt'] = human_data['prompt'].str.replace('n t ', "n't ", regex=False)
human_data['utterance'] = human_data['utterance'].str.replace(r'n t[$\.]', "n't", regex=True)
human_data['prompt'] = human_data['prompt'].str.replace(r'n t[$\.]', "n't", regex=True)
# human_data['prompt'] = human_data['prompt'].str.replace('wasnt', "wasn't", regex=True)
# human_data['utterance'] = human_data['utterance'].str.replace('wasnt', "wasn't", regex=True)

human_utterances = human_data.utterance.apply(mwu_measures.process_text)
human_prompts = human_data.prompt.apply(mwu_measures.process_text)


In [160]:
human_text = pd.concat([human_utterances, human_prompts])
human_text = human_text.explode()
human_text = human_text.dropna()

In [161]:
human_text = human_text.drop_duplicates().dropna().reset_index(drop=True)

In [162]:
text_batches = [human_text.iloc[i*50000:(i+1)*50000].copy() for i in range(int(len(human_text) / 50000) + 1)]

In [ ]:
text_batches = [batch.to_list() for batch in text_batches]
# TODO: does wasn t

In [ ]:
len(text_batches)

In [167]:
test_weights = {'token_freq': 1/8, 'dispersion': 1/4, 'type_1': 1/16, 'type_2': 1/8, 'entropy_1': 1/16, 'entropy_2': 1/8, 'fw_assoc': 1/8, 'bw_assoc': 1/16}

In [ ]:
human_text_mwu = []
i = 0
for batch in text_batches:
    print((i / len(text_batches)))
    human_text_mwu.append(helper.get_score_batch(batch, weights=test_weights, from_text=True))
    i += 1

In [ ]:
human_text_mwu

In [ ]:
pd.concat((result[0] for result in human_text_mwu)).sort_values(by = 'ngram').to_csv('human_mwu.csv', index=False)

In [ ]:
human_text_mwu.drop_duplicates().sort_values(by=['mwu_score'], ascending=False)

In [ ]:
# LO HICE!!!
this_corpus.create_query(ngrams, 'big_1', 'ug_3')
this_corpus.get_ngram_scores('big_1', 'ug_3', 3, [-0.1, 0.1])['normalized']

In [ ]:
this_corpus.df("SELECT * FROM trigram_db WHERE big_1 = HASH('read up') AND ug_3 = HASH('about')")

In [ ]:
helper("SELECT DISTINCT * FROM entropy_diffs", True)

In [ ]:
# entropy broken for trigrams again

In [ ]:
x = helper.get_score_batch(mwu_examples['Expression']).sort_values(by='mwu_score')
x[x['ngram_length'].isna()]

In [ ]:
helper("SELECT * FROM raw_measures", df = True)

In [7]:
ngrams = ['come in handy', 'family friend', 'database management system', 'one hundred percent', 'line of control', 'like a', 'boiled potatoes']

In [4]:
self = helper
normalized=True
# self.get_score_batch(ngrams)

In [5]:
sentence = 'Hello! Come in and eat boiled potatoes they are one hundred percent good.\nI remember when we met in the year 2000'
sentence = mwu_measures.process_text(sentence)

In [6]:
self.create_scores(sentence)

In [ ]:
self.get_score_batch(sentence)
# TODO: why are they repeated?

In [ ]:
this_corpus.df("VACUUM ANALYZE")

In [ ]:
# TODO: Rewrite documentation and merge onto main. This is basically ready.
# TODO: simplify process_text flowline. default for helper?? process > create > return? > clean?
# TODO: Brown as default corpus.
# > this for after cogsci
# TODO: 4-grams. Should need minimal modification. Challenge might be RAM. Consider implementing an option to work from disk with duckdb?
# > This for after cogsci

In [7]:
gpt_data = pd.read_csv('2GPTEmpathicDialoguesDataset (1).csv')

In [8]:
gpt_utterances = gpt_data.processed.apply(mwu_measures.process_text)
gpt_prompts = gpt_data.prompt.apply(mwu_measures.process_text)

In [ ]:
gpt_text = pd.concat([gpt_utterances, gpt_prompts])
gpt_text = gpt_text.explode()
gpt_text = gpt_text.dropna()
gpt_text = gpt_text.drop_duplicates().dropna().reset_index(drop=True)
text_batches = [gpt_text.iloc[i*50000:(i+1)*50000].copy() for i in range(int(len(gpt_text) / 50000) + 1)]
text_batches = [batch.to_list() for batch in text_batches]
nice_weights = {'token_freq': 1/8, 'dispersion': 1/4, 'type_1': 1/16, 'type_2': 1/8, 'entropy_1': 1/16, 'entropy_2': 1/8, 'fw_assoc': 1/8, 'bw_assoc': 1/16}
gpt_text_mwu = []
i = 0
for batch in text_batches:
    print(round(i / len(text_batches), 2))
    gpt_text_mwu.append(helper.get_score_batch(batch, weights=nice_weights, from_text=True))
    i += 1

pd.concat((result[0] for result in gpt_text_mwu)).sort_values(by = 'ngram').to_csv('gpt_mwu.csv', index=False)

In [32]:
human_data_1 = pd.read_csv('test.csv', usecols=['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'])
human_data_2 = pd.read_csv('train.csv', usecols=['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'])
human_data_3 = pd.read_csv('valid.csv', usecols=['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'])
human_data = pd.concat([human_data_1, human_data_2, human_data_3]).reset_index(drop=True)
human_data['utterance'] = human_data['utterance'].str.replace('_comma_', ' , ')
human_data['prompt'] = human_data['prompt'].str.replace('_comma_', ' , ')
human_data['utterance'] = human_data['utterance'].str.replace('n t ', "n't ", regex=False)
human_data['prompt'] = human_data['prompt'].str.replace('n t ', "n't ", regex=False)
human_data['utterance'] = human_data['utterance'].str.replace(r'n t[$\.]', "n't", regex=True)
human_data['prompt'] = human_data['prompt'].str.replace(r'n t[$\.]', "n't", regex=True)
# human_data['prompt'] = human_data['prompt'].str.replace('wasnt', "wasn't", regex=True)
# human_data['utterance'] = human_data['utterance'].str.replace('wasnt', "wasn't", regex=True)

human_data['utterance'] = human_data.utterance.apply(mwu_measures.process_text)

In [33]:
human_scores = pd.read_csv('human_mwu_scores.csv')

In [34]:
human_scores = dict(zip(human_scores['ngram'].to_list(), human_scores['mwu_score'].to_list()))

In [35]:
human_data = human_data.explode('utterance')

In [39]:
human_data = human_data[~human_data['utterance'].isna()]

In [40]:
human_data['mwu_score'] = human_data.utterance.apply(lambda x: human_scores[x])

In [45]:
human_data = human_data.reset_index(drop=True)
human_data['trigram_id'] = human_data.index // 2

In [49]:
human_data[['conv_id', 'utterance_idx', 'prompt', 'speaker_idx', 'utterance', 'mwu_score', 'trigram_id']].to_csv('human_trigram_data.csv', index=False)

Index([      0,       0,       1,       1,       2,       2,       3,       3,
             4,       4,
       ...
       1626548, 1626549, 1626549, 1626550, 1626550, 1626551, 1626551, 1626552,
       1626552, 1626553],
      dtype='int64', length=3253107)

In [50]:
gpt_data = pd.read_csv('2GPTEmpathicDialoguesDataset (1).csv')
gpt_data['utterance'] = gpt_data.processed.apply(mwu_measures.process_text)

In [55]:
gpt_scores = pd.read_csv('gpt_mwu_scores.csv')
gpt_scores = dict(zip(gpt_scores['ngram'].to_list(), gpt_scores['mwu_score'].to_list()))

In [53]:
gpt_data = gpt_data.explode('utterance')

In [54]:
gpt_data = gpt_data[~gpt_data['utterance'].isna()]

In [56]:
gpt_data['mwu_score'] = gpt_data.utterance.apply(lambda x: gpt_scores[x])

In [57]:
gpt_data = gpt_data.reset_index(drop=True)
gpt_data['trigram_id'] = gpt_data.index // 2

In [59]:
gpt_data

,Unnamed: 0,conv_id,context,prompt,gptgen,processed,utterance,mwu_score,trigram_id
0,0,hit:0_conv:1,sentimental,I remember going to the fireworks with my best...,assistant1: I know exactly what you mean. Ther...,i know exactly what you mean. there's somethi...,i know,0.492501,0
1,0,hit:0_conv:1,sentimental,I remember going to the fireworks with my best...,assistant1: I know exactly what you mean. Ther...,i know exactly what you mean. there's somethi...,i know exactly,0.416521,0
2,0,hit:0_conv:1,sentimental,I remember going to the fireworks with my best...,assistant1: I know exactly what you mean. Ther...,i know exactly what you mean. there's somethi...,know exactly,0.511043,1
3,0,hit:0_conv:1,sentimental,I remember going to the fireworks with my best...,assistant1: I know exactly what you mean. Ther...,i know exactly what you mean. there's somethi...,know exactly what,0.574656,1
4,0,hit:0_conv:1,sentimental,I remember going to the fireworks with my best...,assistant1: I know exactly what you mean. Ther...,i know exactly what you mean. there's somethi...,exactly what,0.597637,2
...,...,...,...,...,...,...,...,...,...
11805080,19532,hit:9_conv:19,embarrassed,one time when I was in preschool_comma_ I had ...,"assistant1: Listener: Oh no, that sounds reall...","oh no, that sounds really embarrassing! how ...",more comfortable,0.571433,5902540
11805081,19532,hit:9_conv:19,embarrassed,one time when I was in preschool_comma_ I had ...,"assistant1: Listener: Oh no, that sounds reall...","oh no, that sounds really embarrassing! how ...",more comfortable and,0.479777,5902540
11805082,19532,hit:9_conv:19,embarrassed,one time when I was in preschool_comma_ I had ...,"assistant1: Listener: Oh no, that sounds reall...","oh no, that sounds really embarrassing! how ...",comfortable and,0.506829,5902541
11805083,19532,hit:9_conv:19,embarrassed,one time when I was in preschool_comma_ I had ...,"assistant1: Listener: Oh no, that sounds reall...","oh no, that sounds really embarrassing! how ...",comfortable and confident,0.459067,5902541


In [60]:
gpt_data[['conv_id', 'prompt', 'utterance', 'mwu_score', 'trigram_id']].to_csv('gpt_trigram_data.csv', index=False)